In [1]:
from myml.nbinit import *

In [ ]:
import re
from etl.files import yaml_dump

def dump_yaml_with_anchors(data):
    """
    Dump a dictionary to a YAML string, converting definition keys to anchors
    and replacing quoted alias strings with YAML aliases.

    Args:
        data (dict): The dictionary to dump.

    Returns:
        str: The YAML string with anchors and aliases.
    """
    # Dump the dict to a YAML string. Using default_flow_style=False to get block style.
    dumped = yaml_dump(data)

    # For any key in the definitions block starting with "def_",
    # insert an anchor. This regex finds lines with an indented key that starts with def_.
    dumped = re.sub(
        r"^(\s+)(def_[^:]+):(.*)$",
        lambda m: f"{m.group(1)}{m.group(2)}: &{m.group(2)}{m.group(3)}",
        dumped,
        flags=re.MULTILINE
    )

    # Replace quoted alias strings like '*def_2329260084214905053'
    # with an unquoted alias *def_2329260084214905053.
    dumped = re.sub(
        r"""(['"])(\*def_[^'"]+)\1""",
        lambda m: m.group(2),
        dumped
    )

    return dumped

In [ ]:
import yaml
from collections import defaultdict
from etl.collection.explorer.migration import migrate_csv_explorer
from etl.files import yaml_dump
from etl.paths import EXPLORERS_DIR, STEP_DIR

config = migrate_csv_explorer("habitat-loss")

In [ ]:
# dump explorer config
path_new = STEP_DIR / "export/explorers/biodiversity/latest/habitat_loss.config.yml"
with open(path_new, "w") as f:
    f.write(dump_yaml_with_anchors(config))